### Manual correction of flags, and filtering of the data

Copyright &copy; 2024 Praneeth Vadlapati

In [1]:
import math
import pandas as pd
import gradio as gr
from common_functions import get_latest_filename, safe_flag, \
			harm_categories, unwanted_flags, get_bot_response, \
			print_progress, print_error

UPDATE_MODE = False
admin_flag_path = get_latest_filename('admin-flag', empty_ok=True)
flagged_file_path = get_latest_filename('flagged')
filtered_data_filename = get_latest_filename('filtered')
shortened_text_filename = get_latest_filename('shortened')

flagged_df = pd.read_csv(flagged_file_path)

try:
	admin_flag_df = pd.read_csv(admin_flag_path)
	if UPDATE_MODE:  # update using flag data which might be updated
		# Clear existing flags in admin df and update with new flags
		flags_to_clear = ['text_unsafe', 'flags', 'flag_reason_short']
		admin_flag_df.drop(columns=flags_to_clear, errors='ignore', inplace=True)
		admin_flag_df.update(flagged_df[flags_to_clear])
except FileNotFoundError:
	# If admin file is not found, create a new one
	admin_flag_df = flagged_df.copy()
	admin_flag_df.drop(columns=['url', 'domain_unsafe', 'domain_unindexed'],
						errors='ignore', inplace=True)
	admin_flag_df['admin_unsafe_tags'] = None
	admin_flag_df['admin_unwanted_flags'] = None
	admin_flag_df['admin_flag_reason'] = None
	admin_flag_df.to_csv(admin_flag_path, index=False)

row_count = len(admin_flag_df)
admin_flag_df.head(2)

,id,text,text_unsafe,flags,flag_reason_short,admin_unsafe_tags,admin_unwanted_flags,admin_flag_reason
0,<urn:uuid:faff9b64-041c-4b98-8be4-7ff2a02e4b8d>,We want to know how to best serve you. Please ...,safe,safe,"Report the feedback or report, discrimination,...",safe,safe,"Report the feedback or report, discrimination,..."
1,<urn:uuid:77695799-0774-42a1-8eaa-5efbe154c4e0>,Architectural Control Committee Policies and F...,safe,safe,"Home renovations, mailboxes, and shed guidelin...",safe,safe,"Home renovations, mailboxes, and shed guidelin..."


## Manually correcting the flags

In [ ]:
def isna(val):
	if not val:
		return True
	if isinstance(val, float) and math.isnan(val):
		return True
	if pd.isna(val):
		return True
	return False

# find first row with missing flags
for current_index in range(row_count):
	if isna(admin_flag_df.at[current_index, 'admin_unsafe_tags']) \
			or isna(admin_flag_df.at[current_index, 'admin_unwanted_flags']):
		break

# iterated until last row no missing flags
if current_index == row_count-1 \
		and not isna(admin_flag_df.at[current_index, 'admin_unsafe_tags']) \
		and not isna(admin_flag_df.at[current_index, 'admin_unwanted_flags']):
    current_index = row_count+1


def df_preview():
	start_index = max(current_index - 2, 0)
	end_index = min(current_index + 3, row_count)
	df_display = admin_flag_df.iloc[start_index:end_index].copy()

	df_display.drop(columns=['id', 'text_unsafe', 'flags', 'flag_reason_short'], errors='ignore', inplace=True)
	df_display['text'] = df_display['text'].str.slice(0, 100).replace('\n', '<br>') + '...'
	return df_display.to_markdown()

def get_current_value(column, original_column, return_string=False):
	val = (admin_flag_df.at[current_index, column] if not isna(admin_flag_df.at[current_index, column]) else
		admin_flag_df.at[current_index, original_column] if not isna(admin_flag_df.at[current_index, original_column]) else
		None)
	val = val.split(',') if val else []
	if return_string:
		return ','.join(val)
	return val

def get_current_harmful_values():
	keys = get_current_value('admin_unsafe_tags', 'text_unsafe')
	if safe_flag in keys:
		keys.remove(safe_flag)
	return [f'{key}: {harm_categories.get(key.strip(), True)}' for key in keys]

In [ ]:
editing_mode = True

def evaluate_text(harmful_flags_input, unwanted_flags_input, flag_reason_short):
	global current_index
	if current_index >= row_count:
		return '## Evaluation complete!', df_preview(), gr.update(interactive=False), \
				gr.update(interactive=False), gr.update(interactive=False)

	if harmful_flags_input:
		harmful_flags_input = [flag.strip() for flag in harmful_flags_input]
		harmful_flags_input = [flag.split(':')[0].strip() for flag in harmful_flags_input]
		harmful_flags_input = [flag for flag in harmful_flags_input if flag in harm_categories]
		admin_flag_df.at[current_index, 'admin_unsafe_tags'] = ','.join(harmful_flags_input) or safe_flag
	else:
		admin_flag_df.at[current_index, 'admin_unsafe_tags'] = safe_flag

	if unwanted_flags_input:
		unwanted_flags_input = [flag.strip() for flag in unwanted_flags_input]
		unwanted_flags_input = [flag for flag in unwanted_flags_input if flag in unwanted_flags]
		admin_flag_df.at[current_index, 'admin_unwanted_flags'] = ','.join(unwanted_flags_input) or safe_flag
	else:
		admin_flag_df.at[current_index, 'admin_unwanted_flags'] = safe_flag
	admin_flag_df.at[current_index, 'admin_flag_reason'] = flag_reason_short or ''

	admin_flag_df.to_csv(admin_flag_path, index=False)
	current_index += 1

	if current_index >= row_count:
		# Save backup. Human effort can't be reproduced easily and shouldn't be lost.
		admin_flag_df.to_csv(admin_flag_path+'.bak', index=False)
		# reload page
		return '## Evaluation complete!', df_preview(), gr.update(interactive=False), \
				gr.update(interactive=False), gr.update(interactive=False)

	return f'Text {current_index}: {admin_flag_df.at[current_index, "text"]}', df_preview(), \
			get_current_harmful_values(), get_current_value('admin_unwanted_flags', 'flags'), \
			get_current_value('admin_flag_reason', 'flag_reason_short', return_string=True)


def editing():
	global editing_mode, current_index
	editing_mode = not editing_mode
	current_index = 0


with gr.Blocks() as app:
	gr.Markdown('# Unwanted Text Flags Evaluation')
	if current_index >= row_count:
		gr.Markdown('## Evaluation complete!')
		df_output = gr.Markdown(label='DataFrame Preview', value=df_preview())
		edit_button = gr.Button('Edit')
		edit_button.click(
			editing,
		)

	else:
		text_output = gr.Markdown(value=f'Text {current_index}: {admin_flag_df.at[current_index, "text"]}')

		harm_categories_keys = [f'{key}: {value}' for key, value in harm_categories.items()]
		harmful_flags_input = gr.Dropdown(label='Is the text unsafe?',
			choices=harm_categories_keys, multiselect=True, 
			value=get_current_harmful_values(),
		)
		unwanted_flags_input = gr.Dropdown(label='Unwanted flags?',
			choices=unwanted_flags, multiselect=True, 
			value=get_current_value('admin_unwanted_flags', 'flags'),
		)
		flag_reason_short = gr.Textbox(label='Flag reason (short)',
			value=get_current_value('admin_flag_reason', 'flag_reason_short', return_string=True),
		)
		submit_button = gr.Button('Submit')
		df_output = gr.Markdown(label='DataFrame Preview', value=df_preview())
		submit_button.click(
			evaluate_text, inputs=[harmful_flags_input, unwanted_flags_input, flag_reason_short], 
			outputs=[text_output, df_output, harmful_flags_input, unwanted_flags_input, flag_reason_short]
		)

app.launch()

## Copying admin flags to filtered file

In [2]:
# Integrating admin_df columns into flagged df using ID - admin_unsafe_tags, admin_unwanted_flags

columns_to_copy = {
	'admin_unsafe_tags': 'text_unsafe',  # original column
	'admin_unwanted_flags': 'flags',
	# 'admin_flag_reason': 'flag_reason_short'
}

for admin_column, original_column in columns_to_copy.items():
	# copy admin flags to flagged df
	flagged_df[original_column] = admin_flag_df[admin_column]

flagged_df.drop(columns=['flag_reason_short'], errors='ignore', inplace=True)

flagged_df.to_csv(flagged_file_path, index=False)
flagged_df.head(2)

,id,url,text,domain_unsafe,domain_unindexed,text_unsafe,flags
0,<urn:uuid:faff9b64-041c-4b98-8be4-7ff2a02e4b8d>,http://38.paulosimoes.net/forms/feedback,We want to know how to best serve you. Please ...,NaN,NaN,safe,safe
1,<urn:uuid:77695799-0774-42a1-8eaa-5efbe154c4e0>,http://aberdeencreekfl.com/ACCBusiness/Procedu...,Architectural Control Committee Policies and F...,True,NaN,safe,safe


## Filtering using flagged data

In [3]:
filtered_df = flagged_df.copy()

filtered_df.drop(columns=['url'], errors='ignore', inplace=True)

columns_to_use = ['text_unsafe', 'domain_unsafe', 'domain_unindexed', 'flags']

# replace some strings with booleans
filtered_df[columns_to_use] = filtered_df[columns_to_use].replace({
	'False': None, 'false': None, False: None,
	'True': True, 'true': True, 'Safe': None, 'safe': None, 
	'None': None, 'none': None, '': None,
})

# Print value counts for unsafe tags and transform using harm categories
unsafe_count = filtered_df['text_unsafe'].str.split(',').explode().str.strip().value_counts()
unsafe_count_transformed = unsafe_count.rename(index=harm_categories)
print(unsafe_count_transformed)
print('')

# print count of each value in unwanted flags. if it has multiple values, take first value
flags_count = filtered_df['flags'].str.split(',').explode().str.strip().value_counts()
print(flags_count)
print('')

removal_reason_data = {}  # 'text_unsafe': 10, ...
removed_rows = 0

# drop if any flag is not None
# filtered_df = filtered_df[~filtered_df[columns_to_remove].any(axis=1)]
# filter using each column
for column in columns_to_use:
	removal_count = filtered_df[column].notna().sum()
	removed_rows += removal_count
	print(f'{column}: {removal_count}')
	filtered_df = filtered_df[filtered_df[column].isna()]
filtered_df.drop(columns=columns_to_use, inplace=True)
filtered_df.reset_index(drop=True, inplace=True)
print(f'Removed rows: {flagged_df.shape[0] - filtered_df.shape[0]} of {flagged_df.shape[0]}')
print(f'Retained rows: {filtered_df.shape[0]}')

filtered_df.to_csv(filtered_data_filename, index=False)
filtered_df.head(2)

text_unsafe
Specialized Advice    4
Sexual Content        3
Sex-Related Crimes    1
Privacy               1
Non-Violent Crimes    1
Name: count, dtype: int64

flags
garbage            35
advertisement       9
sensitive_topic     3
biased              3
religious           1
Name: count, dtype: int64

text_unsafe: 8
domain_unsafe: 3
domain_unindexed: 5
flags: 42
Removed rows: 58 of 100
Retained rows: 42


,id,text
0,<urn:uuid:faff9b64-041c-4b98-8be4-7ff2a02e4b8d>,We want to know how to best serve you. Please ...
1,<urn:uuid:76d0f406-290e-41c9-a4bc-2062e6fc6296>,Welcome to AnnieMation’s webpage. We are an in...


## Optimize text for fine-tuning

In [5]:
columns_to_keep = ['text', 'id', 'date']
# Keep only the columns in `columns_to_keep` that are also in `flagged_df`
columns_to_keep = [col for col in columns_to_keep if col in filtered_df.columns]
filtered_df = filtered_df[columns_to_keep]

# consider only columns - text
try:
	short_text_df = pd.read_csv(shortened_text_filename)
	new_filtered_df = filtered_df[columns_to_keep].copy()

	# keep rows that are in filtered_df and remove others
	short_text_df = short_text_df[short_text_df['id'].isin(new_filtered_df['id'])]
	# add missing rows from filtered_df
	missing_rows = new_filtered_df[~new_filtered_df['id'].isin(short_text_df['id'])]
	if not missing_rows.empty:
		short_text_df = pd.concat([short_text_df, missing_rows])
		short_text_df = short_text_df.drop_duplicates(subset='id')
except FileNotFoundError:
	short_text_df = filtered_df[columns_to_keep].copy()
	short_text_df['finetune_text'] = None

# take each row and ask groq to shorten the text and make it suitable for fine-tuning dataset
shortener_prompt_template = (
	'You are a content moderator who is preparing a dataset for fine-tuning a language model. '
	'You have a text that needs to be shortened and made suitable for the dataset. \n'
	'Return the optimized text in the triple backticks. '
	'Retain important details like Date and Location. \n'
	'Original text: ```\n{initial_text}\n```'
)

def get_shorter_text(text, max_retries=3):
	for _ in range(max_retries):
		try:
			response = get_bot_response(messages=[
				{ 'role': 'user', 'content': shortener_prompt_template.format(initial_text=text) }
			])
			# replace single backticks with triple backticks
			if '```' not in response:
				response = response.replace('`', '```')
			response = response.replace('```\n```', '```')
			# get the value from triple backticks
			response = response.split('```')[1].strip()
			if response:
				return response
			else:
				raise Exception('Empty response')
		except Exception as e:
			print(f'Error: {e}. Retrying')

def shorten_text_df(df):
	# if text column is None, get shortened text using initial text
	initial_length_sum = 0
	shortened_length_sum = 0
	for i, row in df.iterrows():
		if pd.isna(row['finetune_text']):
			shorter_text = get_shorter_text(row['text'])
			if not shorter_text or len(shorter_text) > len(row['text']):
				print_error()
				continue
			df.loc[i, 'finetune_text'] = shorter_text
			initial_length_sum += len(row['text'])
			shortened_length_sum += len(shorter_text)
			print_progress()

	saved_length = initial_length_sum - shortened_length_sum
	if initial_length_sum and saved_length:
		saved_percent = (saved_length / initial_length_sum) * 100
		print(f'\nReduced: {saved_length}/{initial_length_sum} characters ({saved_percent:.2f}%)')
	return df


shorten_text_df(short_text_df)
short_text_df.to_csv(shortened_text_filename, index=False)
print(f'Shortened text data size: {short_text_df.shape}')
short_text_df.head(2)

..............Error: list index out of range. Retrying
............................
Reduced: 100790/141905 characters (71.03%)
Shortened text data size: (42, 3)


,text,id,finetune_text
0,"Realme 12 Pro Plus: In the coming new year, al...",<urn:uuid:f65537ba-005c-40d2-be47-432b9c558a71>,Realme 12 Pro Plus: Realme is set to launch it...
1,Yesterday was a day with no apparent progress....,<urn:uuid:3bb2ccac-eb5e-4827-b321-95ca06142138>,"Yesterday was a day with no apparent progress,..."
